In [1]:
from utils import get_webpage_content, extract_params, process_model_output, filter_json
from json5 import loads
import pandas as pd
pd.set_option('display.max_colwidth', None)
from tqdm import tqdm

In [ ]:
example_single = "I'm searching for beers with ABV greater than 6%, IBU less than 80, EBC greater than 15, brewed after 01-2025, using yeast named Brettanomyces, hops named Nugget, malt named Chocolate, that go well with fish_and_chips, and named Punk_IPA."
processed_text = extract_params(example_single)
# print('*' + processed_text)
processed_text = process_model_output(processed_text)
# print('**' + processed_text)
processed_text = loads(processed_text)
print(str(processed_text))
print('-----------------')
processed_text_filtered, errors = filter_json(processed_text)
print(str(processed_text_filtered), errors)

In [2]:
with open('test_cases.txt', 'r') as f:
    examples = f.read().split('\n')

examples_processed = [example.split("\"")[1].split("\"")[0] for example in examples]
examples_cases = [example.split("(")[1].split(")")[0] if len(example.split("(")) > 1 else None for example in examples]

In [3]:
examples_list = []
case_list = []
output_list = []
output_filtered_list = []
error_list = []

for i in tqdm(range(5)):
    examples_list.append(examples_processed[i])
    case_list.append(examples_cases[i])
    count = 0
    while count < 2:
        try:
            processed_text = loads(process_model_output(extract_params(examples_processed[i])))
            processed_text_filtered, errors = filter_json(processed_text)
            output_list.append(processed_text)
            output_filtered_list.append(processed_text_filtered)
            error_list.append(errors)
            count = 2
        except:
            count += 1
            if count == 2:
                output_list.append("{}")
                output_filtered_list.append("{}")
                error_list.append("")
            continue

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
pd.DataFrame({"Example": examples_list, "Edge_Case": case_list, "Output": output_list, "Filtered_Output": output_filtered_list, "Errors": error_list})

In [ ]:
query_string = "&".join([f"{key}={value}" for key, value in output_filtered_list.items()] + [f"per_page={per_page}"])

In [ ]:
query_string

In [ ]:
content = get_webpage_content("https://api.punkapi.com/v2/beers?" + query_string)
content = loads(content)

In [ ]:
len(content)

In [ ]:
content

In [ ]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
tokenizer = openai.